In [1]:
import sys
import os
import importlib

repo_root = os.path.dirname(os.getcwd())
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)


In [2]:
from io_load import *
txn = load_transactions_csv()
rc = load_receipts_csv()
visit = load_users_logins_csv()

In [3]:
from clean_compress import * 
tuple_3df = drop_duplicate_rows(txn, rc, visit) 
txn = reduce_mem_usage(tuple_3df[0])
rc = reduce_mem_usage(tuple_3df[1])
visit = reduce_mem_usage(tuple_3df[2])


Mem. usage decreased to 89.79 Mb (18.7% reduction)
Mem. usage decreased to 629.46 Mb (22.4% reduction)
Mem. usage decreased to 42.96 Mb (0.0% reduction)


In [5]:
# reassign User_id:
tuple_3df, _ = id_reassign_with_map(["User_id"], txn, rc, visit)
# reassign Shop_id, Order_id:
tuple_1df, _ = id_reassign_with_map(["Shop_id", "Order_id"], tuple_3df[0])
# reassign Coupon_id:
tuple_2df, _ = id_reassign_with_map(["Coupon_id"], tuple_1df[0], tuple_3df[1]) 
# final assignment
txn = tuple_2df[0]
rc = tuple_2df[1]
visit = tuple_3df[2]


[User_id] ↓ to 130.10 MB (90.1% reduction)
[Shop_id] ↓ to 11.78 MB (90.1% reduction)
[Order_id] ↓ to 11.78 MB (90.1% reduction)
[Coupon_id] ↓ to 108.62 MB (89.9% reduction)


In [7]:
tuple_3df = drop_original_id(["User_id"], txn, rc, visit)
tuple_1df = drop_original_id(["Shop_id", "Order_id"], tuple_3df[0])
tuple_2df = drop_original_id(["Coupon_id"], tuple_1df[0], tuple_3df[1])
txn = tuple_2df[0]
rc = tuple_2df[1]
visit = tuple_3df[2]

In [11]:
save_df2pq(txn, "txns")
save_df2pq(rc, "rcs")
save_df2pq(visit, "visits")

[txns] saved to c:\Users\20878\25Fall\meituan-coupon-roi\data_work/txns.parquet, shape=(1543536, 9), size=26.79 MB
[rcs] saved to c:\Users\20878\25Fall\meituan-coupon-roi\data_work/rcs.parquet, shape=(12693100, 8), size=147.92 MB
[visits] saved to c:\Users\20878\25Fall\meituan-coupon-roi\data_work/visits.parquet, shape=(2815177, 2), size=8.59 MB


In [2]:
from io_load import *
txn = load_df_from_pq(os.path.join(repo_root, "data_work/txns.parquet"))
rc = load_df_from_pq(os.path.join(repo_root, "data_work/rcs.parquet"))

pq loaded from c:\Users\20878\25Fall\meituan-coupon-roi\data_work/txns.parquet, shape=(1543536, 9), size=26.79 MB
pq loaded from c:\Users\20878\25Fall\meituan-coupon-roi\data_work/rcs.parquet, shape=(12693100, 8), size=147.92 MB


In [3]:
from reconcile import *
txn, rc = add_keys(txn, rc)
save_df2pq(txn, "txns_keys")
save_df2pq(rc, "rcs_keys")

[txns_keys] saved to c:\Users\20878\25Fall\meituan-coupon-roi\data_work/txns_keys.parquet, shape=(1543536, 10), size=33.21 MB
[rcs_keys] saved to c:\Users\20878\25Fall\meituan-coupon-roi\data_work/rcs_keys.parquet, shape=(12693100, 9), size=199.79 MB
